In [37]:
from qiskit import QuantumCircuit, transpile
from qiskit.primitives import BackendSamplerV2
from sympy import gcd, mod_inverse # 주기 찾기 후 고전적 계산에 사용
from qiskit_aer import AerSimulator # 시뮬레이션 백엔드
from fractions import Fraction # 측정 결과 위상값을 분수로 근사하는 데 사용
from random import randint # 무작위로 'a'를 선택하는 데 사용
from qiskit.circuit.library import QFT # 양자 푸리에 변환 라이브러리
import math # ceil 및 log2 함수 사용을 위해 math 모듈 임포트

def c_amod15(a, power):
    """
    N=15에 대한 a^power mod 15 연산을 수행하는 제어된 양자 회로를 생성합니다.
    이 함수는 N=15에 대해 gcd(a, 15)=1 인 a (2, 7, 8, 11, 13)에 대해서만 유효합니다.
    power는 2^x 형태의 값이 들어옵니다.
    """
    if a not in [2, 7, 8, 11, 13]:
        raise ValueError("'a' must be 2, 7, 8, 11, or 13 for N=15")

    U = QuantumCircuit(4) # 보조 큐비트 4개에 대한 연산 (N=15에 대해 필요)

    # U_a 연산을 power번 반복합니다. power는 2^x 형태로 주어집니다.
    # U_a 연산은 N=15와 특정 a 값의 성질을 이용한 최적화된 스왑(swap) 게이트 조합으로 구현됩니다.
    for iteration in range(power):
        if a in [2, 13]:
            U.swap(0, 1)
            U.swap(1, 2)
            U.swap(2, 3)
        elif a in [7, 8]:
            U.swap(2, 3)
            U.swap(1, 2)
            U.swap(0, 1)
        elif a == 11:
            U.swap(1, 3)
            U.swap(0, 2)
        # Note: a=4의 경우는 gcd(4, 15) = 1 이 아니므로 초기 a 선택 단계에서 걸러집니다.
        # 다른 a 값에 대한 U_a 연산은 더 복잡한 회로가 필요할 수 있습니다.

    U = U.to_gate() # 서브 회로를 단일 게이트로 만듭니다.

    # 제어된 게이트로 만듭니다. 제어 큐비트는 외부에서 연결될 것입니다.
    # 이 게이트는 제어 큐비트 1개와 타겟 큐비트 4개(보조 큐비트)를 가집니다.
    c_U = U.control()
    return c_U


def controlled_modular_exponentiation_N15(a, N, n_count):
    """
    N=15에 대한 제어된 모듈러 지수 연산 U_a^x mod N의 양자 회로를 생성합니다.
    이 함수는 N=15에 대해서만 유효합니다.
    실제 U_a^(2^x) 연산 회로를 생성합니다.
    """
    if N != 15:
        raise ValueError("This controlled_modular_exponentiation_N15 function is only for N=15.")

    # N=15에 필요한 보조 큐비트 수 계산 (ceil(log2(15)) = 4)
    n_aux = int(math.ceil(math.log2(N)))
    if n_aux == 0: n_aux = 1 # N=1인 경우는 없겠지만 안전 장치
    total_qubits = n_count + n_aux # 전체 큐비트 수 (위상 추정 n_count + 보조 n_aux)

    qc_exp = QuantumCircuit(total_qubits) # 위상 추정 큐비트 + 보조 큐비트로 구성된 회로 생성

    # 제어된 모듈러 지수 연산 추가
    # 각 제어 큐비트 x (0부터 n_count-1)에 대해 U_a^(2^x) mod N 연산을 제어되게 적용합니다.
    # 제어 큐비트는 회로 인덱스 range(n_count)에 해당 (위상 추정 큐비트들)
    # 타겟 큐비트는 회로 인덱스 range(n_count, n_count + n_aux)에 해당 (보조 큐비트들)

    for x in range(n_count):
        # c_amod15(a, 2**x)는 a^(2^x) mod 15 연산을 수행하는 제어된 게이트를 반환합니다.
        controlled_U_gate = c_amod15(a, 2**x)

        # 메인 회로에 생성된 제어된 게이트 추가
        # append의 첫 번째 인자는 추가할 게이트,
        # 두 번째 인자는 게이트가 적용될 큐비트들의 인덱스 리스트입니다.
        # controlled_U_gate는 이미 제어 큐비트 1개와 타겟 큐비트 n_aux개를 가지도록 정의되어 있습니다.
        # 따라서 큐비트 인덱스 리스트는 [제어 큐비트 인덱스] + [타겟 큐비트 인덱스들] 형태가 됩니다.
        # 제어 큐비트는 x (0 ~ n_count-1), 타겟 큐비트는 보조 큐비트들입니다.
        target_qubit_indices = list(range(n_count, total_qubits))
        qc_exp.append(controlled_U_gate, [x] + target_qubit_indices)

    return qc_exp


# --- Shor 알고리즘 전체 회로 생성 (N=15에 특화된 모듈러 지수 연산 사용) ---
def create_shor_circuit_N15(N, a, n_count):
    """
    주어진 N=15와 a에 대해 Shor 알고리즘의 양자 주기 찾기 회로를 생성합니다.
    N=15에 특화된 모듈러 지수 연산 구현을 사용합니다.
    """
    if N != 15:
        raise ValueError("This create_shor_circuit_N15 function is only for N=15.")

    # N=15에 필요한 보조 큐비트 수 계산 (ceil(log2(15)) = 4)
    n_aux = int(math.ceil(math.log2(N)))
    if n_aux == 0: n_aux = 1 # N=1인 경우는 없겠지만 안전 장치
    total_qubits = n_count + n_aux # 전체 큐비트 수

    # 양자 회로 및 클래식 레지스터 생성
    qc = QuantumCircuit(total_qubits, n_count) # 위상 추정 n_count + 보조 n_aux 큐비트, 측정 n_count 클래식 비트

    # 1단계: 위상 추정 큐비트 초기화 (아다마르 변환 적용)
    qc.h(range(n_count))

    # 2단계: 보조 큐비트 초기 상태 설정 (|1> 상태)
    # N=15이고 U_a 연산이 4개의 보조 큐비트에 적용될 때 |1> 상태는 4번째 큐비트(|0001>)에 해당합니다.
    qc.x(n_count + n_aux - 1) # n_count + 3 에 해당 (0-indexed)

    # 3단계: N=15에 특화된 제어된 모듈러 지수 연산 U_a^x mod N 추가
    controlled_exp_circuit = controlled_modular_exponentiation_N15(a, N, n_count)

    # 생성된 제어된 모듈러 지수 연산 회로를 메인 회로에 추가
    # range(total_qubits)는 controlled_exp_circuit이 적용될 메인 회로의 큐비트 인덱스 범위를 의미합니다.
    # controlled_exp_circuit 자체는 total_qubits 만큼의 큐비트를 가지도록 설계되었습니다.
    qc.compose(controlled_exp_circuit, range(total_qubits), inplace=True)


    # 4단계: 역 양자 푸리에 변환 (IQFT) 적용
    # IQFT는 위상 추정 큐비트 (첫 n_count 개)에만 적용됩니다.
    qc.append(QFT(n_count, do_swaps=False).inverse(), range(n_count)) # do_swaps=False: QFT 내 큐비트 스왑 없음

    # 5단계: 위상 추정 큐비트 측정
    # 첫 n_count 개 큐비트를 클래식 레지스터 n_count 개에 측정합니다.
    qc.measure(range(n_count), range(n_count))

    return qc

# --- Shor 알고리즘 실행 및 인수 찾기 ---

# 유효한 인수를 찾을 때까지 반복
factor1 = 1 # 초기값 설정 (반복문 진입 조건 만족: 1*15 != 15 또는 1 또는 15)
factor2 = 15 # 초기값 설정
attempt_count = 0
max_attempts = 20 # 무한 루프 방지를 위한 최대 시도 횟수 설정 (충분히 큰 값)
N = 15

# 루프: 인수를 찾거나 최대 시도 횟수를 초과할 때까지 반복합니다.
while factor1 * factor2 != N or factor1 == 1 or factor2 == 1:
    attempt_count += 1
    if attempt_count > max_attempts:
        print(f"\n최대 시도 횟수 ({max_attempts}) 초과. 인수를 찾지 못했습니다.")
        break # 최대 시도 횟수에 도달하면 루프 종료

    print(f"\n--- 시도 #{attempt_count} ---")

    # 1단계 (고전적 전처리): 2부터 N-1 사이의 a를 선택 (gcd(a, N) == 1이 될 때까지 반복)
    # Shor 알고리즘의 첫 번째 단계로, N과 서로소인 정수 a를 찾습니다.
    a = randint(2, N - 1)
    while gcd(a, N) != 1:
        a = randint(2, N - 1)
    print(f"선택된 a: {a}")

    # 2단계 (양자 주기 찾기): 위상 추정에 사용할 큐비트 수 결정
    # 일반적으로 2*log2(N) + 1 이상을 권장합니다.
    # N=15, log2(15) 약 3.9, 2*3.9+1 = 8.8 -> 9개 이상 권장
    # 시뮬레이터 성능 및 필요 정확도에 따라 조절할 수 있습니다. 여기서는 8개 사용 (예시)
    n_count = 8

    # 양자 회로 생성 (선택된 'a' 값에 따라 회로가 달라집니다.)
    shor_circuit = create_shor_circuit_N15(N, a, n_count)

    # 회로 확인 (선택 사항)
    # print(shor_circuit.draw(output='text')) # 텍스트로 회로 그리기


    # 3단계 (양자 주기 찾기): 회로 실행 및 결과 얻기
    simulator = AerSimulator() # 시뮬레이터 백엔드 생성
    sampler = BackendSamplerV2(backend=simulator) # 샘플러 생성

    # --- 회로 실행 전에 Transpile 과정 추가 ---
    print("회로 Transpile 시작...")
    try:
        # 최종 회로를 AerSimulator에 맞게 Transpile (분해 및 최적화)
        # Transpile 과정에서 커스텀 게이트들이 시뮬레이터의 기반 게이트로 분해됩니다.
        transpiled_shor_circuit = transpile(shor_circuit, backend=simulator, optimization_level=0) # 필요시 optimization_level 조절
        print("회로 Transpile 완료.")
        # print(transpiled_shor_circuit.draw(output='text')) # (선택 사항) Transpile된 회로 확인

        # Transpile된 회로를 Sampler에 전달하여 실행
        print(f"Transpile된 회로 실행 시작 (샷 수: 2048)...")
        job = sampler.run([transpiled_shor_circuit], shots=2048) # Transpile된 회로 사용
        result = job.result() # 결과 대기

        # SamplerV2 결과에서 측정 결과 카운트를 얻는 방법
        # Qiskit 버전에 따라 접근 방식이 조금 다를 수 있습니다. 최신 방식과 이전 방식을 모두 시도합니다.
        counts = None
        try:
            # SamplerV2 최신 접근 방식 (JobV2 결과)
            counts = result[0].data.counts
            # print("SamplerV2 result[0].data.counts로 counts를 가져왔습니다.")
        except Exception as e_v2:
            # 이전 버전 또는 다른 구조일 경우 시도
            # print(f"SamplerV2 result[0].data.counts 시도 중 오류 발생: {e_v2}. 다른 방법을 시도합니다.")
            try:
                # JobV1 또는 다른 결과 구조 접근
                counts = result[0].join_data().get_counts()
                # print("result.get_counts()로 counts를 가져왔습니다.")
            except Exception as e_v1:
                print(f"counts를 가져오는 여러 시도 중 오류 발생: {e_v1}")


        if counts:
            # print(f"측정 결과 카운트: {counts}")

            # 가장 확률 높은 측정값(이진 문자열) 얻기
            measured = max(counts, key=counts.get)
            # print(f"가장 빈번한 측정값 (이진): {measured}")

            # 측정 결과에서 위상 추정값 계산 (0과 1 사이의 실수)
            # measured는 n_count 비트의 이진 정수이므로 2^n_count로 나누어 위상값을 얻습니다.
            phase = int(measured, 2) / (2**n_count)
            print(f"추정된 위상 (s/r): {phase}")

            # 4단계 (고전적 후처리): 분수 근사를 통해 주기 r 찾기
            # 위상 추정값 phase를 분수 s/r 형태로 나타내어 분모 r을 찾습니다.
            # limit_denominator(N)는 분모를 N 이하로 제한하여 의미 있는 분수를 찾습니다.
            frac = Fraction(phase).limit_denominator(N)
            r_candidate = frac.denominator # 분모가 주기 후보입니다.
            print(f"분수 근사를 통해 찾은 주기의 후보 r: {r_candidate}")

            # 5단계 (고전적 후처리): 찾은 주기 r로 인수 계산
            # 찾은 주기가 짝수이고, a^(r/2) mod N != N-1 인 경우에만 인수를 찾을 수 있습니다.
            if r_candidate % 2 != 0:
                print("찾은 주기 r은 홀수입니다. 다음 시도로 넘어갑니다.")
            else:
                # gcd(a^(r/2) + 1, N) 와 gcd(a^(r/2) - 1, N) 계산
                # pow(a, r_candidate // 2, N)는 a^(r_candidate/2) mod N을 효율적으로 계산합니다.
                x = pow(a, r_candidate // 2, N)

                if x == N - 1:
                    print(f"a^(r/2) mod N = {N-1} 입니다. 비자명 인수를 찾지 못했습니다. 다음 시도로 넘어갑니다.")
                else:
                    factor1 = gcd(x + 1, N)
                    factor2 = gcd(x - 1, N)

                    # 찾은 인수가 자명하지 않은 경우 (1이나 N 자신이 아닌 경우)
                    if factor1 * factor2 == N and factor1 != 1 and factor2 != 1:
                        print(f"\n--- 성공! 인수를 찾았습니다. ---")
                        print(f"N={N}의 인수는 {factor1} 및 {factor2} 입니다.")
                        # 루프 종료 조건 만족
                    else:
                        print(f"찾은 주기 r={r_candidate} 로는 비자명 인수를 찾지 못했습니다. 다음 시도로 넘어갑니다.")
                        # print(f"gcd({x}+1, {N}) = {factor1}")
                        # print(f"gcd({x}-1, {N}) = {factor2}")
        else:
            print("회로 실행 결과(counts)를 가져오지 못했습니다. 다음 시도로 넘어갑니다.")

    except Exception as main_loop_e:
        # Transpile 또는 Sampler 실행 자체에서 오류가 발생한 경우
        print(f"메인 루프 실행 중 예상치 못한 오류 발생: {main_loop_e}")
        # 이 시도는 실패한 것으로 간주하고 루프를 계속합니다.
        factor1 = 1
        factor2 = N
        continue


# 루프 종료 후 최종 결과 출력
# factor1 * factor2 == N and factor1 != 1 and factor2 != 1 조건으로 루프가 종료되면 성공입니다.
if factor1 * factor2 == N and factor1 != 1 and factor2 != 1:
    print(f"\nShor 알고리즘 실행 종료: N={N}의 인수는 {factor1} 및 {factor2} 입니다.")
else:
    print(f"\n{N}의 인수를 찾는 데 실패했습니다 (총 시도 횟수: {attempt_count}).")


--- 시도 #1 ---
선택된 a: 14


ValueError: 'a' must be 2, 7, 8, 11, or 13 for N=15